# DKTC (Dangerous Talk Classification)
## AIFFEL DLthon - 한국어 위협 대화 5클래스 분류

---

### 전략: 가브리엘 페터슨 탑다운 접근법

| Level | 전략 | 이 코드에서 해당하는 부분 |
|-------|------|--------------------------|
| **Level 1** | 베이스라인 → 문제 발견 | STEP 0~1: EDA로 "일반대화 0개" 문제 발견, STEP 5 Exp1: CE Loss baseline |
| **Level 2** | 최신 논문 기법으로 해결 | STEP 2: 합성데이터(AugGPT 영감), STEP 4: Focal Loss + R-Drop, STEP 5 Exp2~3 |
| **Level 3** | Ablation Study로 정량 검증 | STEP 5 Exp1~4 비교, STEP 6: 시각화로 기여도 증명 |

### 코드 구조 요약

| STEP | 내용 | 배운 것 / 최신 기법 |
|------|------|---------------------|
| 0 | 환경 설정, GPU 확인 | - |
| 1 | EDA - 클래스 분포, 길이 분석 | Ex06 (네이버 영화리뷰 감성분석) |
| 2 | 일반대화 합성데이터 5개 소스 | AugGPT (Dai et al., 2023) 논문 영감 |
| 3 | 텍스트 전처리 | Ex06 전처리, Ex07 (SentencePiece 토큰화) |
| 4 | KcELECTRA + Focal Loss + R-Drop 정의 | Ex09 (Transformer), ELECTRA/Lin 2017/Liang 2021 논문 |
| 5 | Ablation 4개 실험 실행 | Ex03 (Ablation Study 설계 방법론) |
| 6 | 시각화 (학습곡선, Confusion Matrix) | Ex03 시각화 |
| 7 | 테스트 예측 → submission.csv | - |
| 8 | 프로젝트 정리 | 발표 포인트 |

### 클래스 매핑

| 라벨 | 클래스 |
|------|--------|
| 0 | 협박 대화 |
| 1 | 갈취 대화 |
| 2 | 직장 내 괴롭힘 대화 |
| 3 | 기타 괴롭힘 대화 |
| 4 | 일반 대화 (train에 0개 → 합성 필요!) |

## STEP 0: 환경 설정 및 설치
> **Level 1 (베이스라인 준비)** — 실험에 필요한 라이브러리 설치 및 외부 데이터 다운로드

In [ ]:
# ============================================================
# STEP 0-1: 라이브러리 설치 + 합성데이터 소스 미리 다운로드
# ============================================================
# transformers: HuggingFace 모델/토크나이저 (KcELECTRA 로드용)
# datasets: HuggingFace 데이터셋 (kor_unsmile, NSMC 다운로드용)
# accelerate: HuggingFace 모델 최적화 (transformers 내부 의존성)
!pip install -q transformers datasets accelerate scikit-learn matplotlib seaborn pandas

# [Level 2] 합성데이터 소스 미리 다운로드 (STEP 2에서 사용)
# → AugGPT(Dai et al., 2023) 논문에서 영감: "학습데이터 부족 시 외부 소스로 증강"
# SmileStyle: 스마일게이트 AI가 공개한 한국어 17가지 스타일 대화 데이터셋
!wget -q https://raw.githubusercontent.com/smilegate-ai/korean_smile_style_dataset/main/smilestyle_dataset.tsv -O smilestyle_dataset.tsv
# KakaoChatData: 실제 카카오톡 대화 스타일 챗봇 데이터 (~73K쌍)
!wget -q https://raw.githubusercontent.com/Ludobico/KakaoChatData/main/Dataset/ChatbotData.csv -O ChatbotData.csv

print("설치 완료!")

In [ ]:
# ============================================================
# STEP 0-2: import + 시드 고정 + GPU 확인
# ============================================================
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader          # Ex06에서 배운 Dataset/DataLoader 패턴
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,     # HuggingFace 모델 로드 (Ex09 Transformer 활용)
    get_linear_schedule_with_warmup                        # 학습률 스케줄러: warmup 후 선형 감소
)
from sklearn.model_selection import train_test_split       # stratified 분할 (클래스 비율 유지)
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix  # Ex03 Ablation 평가 지표
)
from datasets import load_dataset                          # HuggingFace Hub에서 데이터 다운로드
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'  # Colab 한글 깨짐 방지

# ──────────────────────────────────────────────
# 재현성(reproducibility) 보장을 위한 시드 고정
# → 같은 시드면 같은 결과가 나옴 → Ablation 비교가 공정해짐
# ──────────────────────────────────────────────
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True  # GPU 연산도 결정론적으로

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# ============================================================
# STEP 0-3: 클래스 매핑 + 하이퍼파라미터 설정
# ============================================================

# 5개 클래스 정의 (대회 제출 형식: 정수 0~4)
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']
CLASS2IDX = {name: idx for idx, name in enumerate(CLASS_NAMES)}  # "협박 대화" → 0
IDX2CLASS = {idx: name for idx, name in enumerate(CLASS_NAMES)}  # 0 → "협박 대화"
NUM_CLASSES = 5

# ──────────────────────────────────────────────
# 하이퍼파라미터
# ──────────────────────────────────────────────
# [Level 2] 모델 선택 근거:
#   KcELECTRA(beomi) = ELECTRA(Clark et al., 2020) 구조를 한국어 댓글/구어체로 사전학습
#   → DKTC 데이터가 반말+구어체이므로 도메인이 일치
#   → 일반 BERT/KoBERT보다 이런 비격식 텍스트에서 성능 우수
MODEL_NAME = 'beomi/KcELECTRA-base-v2022'

MAX_LEN = 256       # 토큰 최대 길이 (STEP 3에서 95%ile 확인 후 결정)
BATCH_SIZE = 16      # GPU 메모리에 맞춤 (T4 기준)
EPOCHS = 5           # 시간 부족 시 3으로 줄여도 됨
LR = 2e-5            # Pre-trained 모델 fine-tuning 표준 학습률
WEIGHT_DECAY = 0.01  # AdamW 정규화 (과적합 방지)
WARMUP_RATIO = 0.1   # 전체 스텝의 10%는 학습률을 서서히 올림 (안정적 시작)
MAX_GRAD_NORM = 1.0  # Gradient Clipping (기울기 폭발 방지)
VAL_RATIO = 0.15     # 15% 검증셋 (stratified로 클래스 비율 유지)

print("설정 완료")
print(f"모델: {MODEL_NAME}")
print(f"EPOCHS: {EPOCHS}, LR: {LR}, MAX_LEN: {MAX_LEN}")

## STEP 1: 데이터 로드 및 EDA
> **Level 1 (베이스라인 → 문제 발견)** — Ex06(네이버 영화리뷰 감성분석)에서 배운 EDA 패턴 적용
>
> 여기서 **핵심 문제를 발견**: train에 "일반 대화" 클래스가 **0개**!
> → test에는 ~100개 있을 것으로 추정 → 이걸 해결 안 하면 일반대화를 절대 맞출 수 없음

In [ ]:
# ============================================================
# STEP 1-1: 데이터 로드
# [Level 1] 가장 먼저 할 일: 데이터를 눈으로 확인하기
# ============================================================
# ⚠️ train.csv, test.csv, submission.csv를 Colab에 업로드하세요!
# 방법 1: 왼쪽 파일 탭에서 드래그 앤 드롭
# 방법 2: 아래 코드 실행
# from google.colab import files
# uploaded = files.upload()

train_df = pd.read_csv('train.csv')      # 3,950개, 4개 클래스만 존재
test_df = pd.read_csv('test.csv')        # 500개, 5개 클래스 (일반대화 포함)
submission_df = pd.read_csv('submission.csv')  # idx + class(예측값 넣을 곳)

print(f"Train: {len(train_df)}개")
print(f"Test:  {len(test_df)}개")
print(f"\n컬럼: {list(train_df.columns)}")
# ⭐ 여기서 핵심 문제 발견: '일반 대화'가 없다!
print(f"\n클래스 분포:")
print(train_df['class'].value_counts())

In [ ]:
# ============================================================
# STEP 1-2: EDA 시각화
# [Level 1] Ex06에서 배운 데이터 탐색 패턴
# → 클래스 분포, 텍스트 길이, 클래스별 길이 차이를 시각화
# → 이 단계에서 "일반 대화 0개" 문제를 눈으로 확인
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1) 클래스 분포 → ⭐ "일반 대화"가 아예 없음을 확인!
class_counts = train_df['class'].value_counts()
axes[0].barh(class_counts.index, class_counts.values, color='steelblue')
axes[0].set_title('Class Distribution (Train)')
axes[0].set_xlabel('Count')
for i, v in enumerate(class_counts.values):
    axes[0].text(v + 10, i, str(v), va='center')  # 막대 끝에 숫자 표시

# 2) 대화 길이 분포 → MAX_LEN 결정에 참고
train_df['text_len'] = train_df['conversation'].str.len()
axes[1].hist(train_df['text_len'], bins=50, color='coral', edgecolor='white')
axes[1].set_title('Conversation Length Distribution')
axes[1].set_xlabel('Characters')
axes[1].axvline(train_df['text_len'].mean(), color='red', linestyle='--',
                label=f"Mean: {train_df['text_len'].mean():.0f}")
axes[1].legend()

# 3) 클래스별 길이 분포 → 클래스 간 길이 차이가 있는지 확인
for cls_name in train_df['class'].unique():
    subset = train_df[train_df['class'] == cls_name]['text_len']
    axes[2].hist(subset, bins=30, alpha=0.5, label=cls_name)
axes[2].set_title('Length by Class')
axes[2].set_xlabel('Characters')
axes[2].legend(fontsize=8)

plt.tight_layout()
plt.show()

print(f"\n대화 길이 통계:")
print(train_df['text_len'].describe())
# ⭐ Level 1 핵심 발견: 이 문제를 Level 2에서 합성데이터로 해결
print(f"\n⚠️ '일반 대화' 클래스가 train에 없음! → 합성 데이터 필요")

## STEP 2: 합성 일반대화 수집 (5개 소스)
> **Level 2 (최신 논문 기법으로 문제 해결)** — AugGPT(Dai et al., 2023) 논문에서 영감
>
> **문제**: train에 일반대화가 0개 → 모델이 "일반 대화"를 전혀 학습할 수 없음
>
> **해결**: 외부 공개 데이터에서 일반대화를 수집하여 합성
> - 논문 근거: AugGPT는 "소규모 데이터를 LLM으로 증강"하는 기법을 제안
> - 우리는 LLM 대신 **공개 데이터셋 5개를 조합**하여 다양성 확보
> - 단일 소스가 아닌 **다중 소스**로 편향(bias)을 줄임

| # | 소스 | 왜 이 소스를 선택했나 | 목표 수 |
|---|------|----------------------|---------|
| 1 | SmileStyle | 17가지 스타일 중 informal(반말) = DKTC 대화체와 가장 유사 | 400 |
| 2 | KakaoChatData | 실제 카톡 대화 → 가장 자연스러운 구어체 | 300 |
| 3 | kor_unsmile | 혐오표현 데이터에서 clean=1(비혐오)만 추출 → "위험하지 않은" 문장 | 200 |
| 4 | NSMC | 긍정 영화리뷰 → 무해한 일상 감정 표현 | 100 |
| 5 | **경계 케이스** ⭐ | "야 죽을래 ㅋㅋ" 같은 농담 → **핵심 차별화 포인트** | 25 |

In [ ]:
# ============================================================
# STEP 2-0: 위협 키워드 필터 정의
# [Level 2] 합성데이터 품질 관리
# → 외부 데이터에서 수집할 때, 실제 위협 문장이 섞이면 라벨 오류 발생
# → 키워드 기반 필터로 위협성 문장을 사전에 제거
# ============================================================

# 4개 위협 클래스의 핵심 키워드를 수집
# 협박: 물리적 위협 ("죽여", "찔러")
# 갈취: 금전 요구 ("돈 내놔", "빚")
# 직장 괴롭힘: 조직 관련 ("해고", "사직서")
# 기타 괴롭힘: 사회적 따돌림 ("따돌", "왕따")
THREAT_KEYWORDS = [
    '죽여', '죽일', '찔러', '칼로', '패줄', '두들겨', '불질러',     # 협박
    '협박', '신고', '경찰', '감옥', '고소', '소송',                   # 협박/법적 위협
    '돈 내놔', '송금', '이자', '빚', '갚아',                          # 갈취
    '해고', '짤리', '사직서', '퇴사', '상사',                         # 직장 괴롭힘
    '따돌', '왕따', '무시', '괴롭'                                    # 기타 괴롭힘
]

def contains_threat(text):
    """텍스트에 위협 키워드가 하나라도 포함되면 True 반환"""
    return any(kw in str(text) for kw in THREAT_KEYWORDS)

normal_samples = []  # 모든 소스에서 수집한 일반대화를 여기에 모음

In [ ]:
# ============================================================
# STEP 2-1: 소스 1 - SmileStyle (목표 400개)
# [Level 2] 스마일게이트 AI 공개 데이터셋
# 선택 이유: 17가지 한국어 스타일 중 informal(반말) 컬럼이
#           DKTC 대화 데이터의 구어체/반말 스타일과 가장 유사
# 처리 방법: 단문 3개를 이어 붙여 멀티턴 대화처럼 구성
#           (DKTC train 평균 ~220자에 맞추기 위해)
# ============================================================
print("="*50)
print("소스 1: SmileStyle 한국어 멀티턴 대화")
print("="*50)

try:
    smile_df = pd.read_csv('smilestyle_dataset.tsv', sep='\t')
    print(f"전체 데이터: {len(smile_df)}개")
    print(f"컬럼: {list(smile_df.columns)}")

    # informal/chat/반말/casual 키워드가 포함된 컬럼 찾기
    # → 이 컬럼이 DKTC의 반말 대화체와 가장 유사
    target_cols = []
    for col in smile_df.columns:
        if any(kw in col.lower() for kw in ['informal', 'chat', '반말', 'casual']):
            target_cols.append(col)

    if not target_cols:
        print(f"  → informal 컬럼 못찾음, 전체 컬럼: {list(smile_df.columns)}")
        target_cols = [smile_df.columns[-1]]

    print(f"  사용 컬럼: {target_cols}")

    smile_texts = []
    for col in target_cols:
        texts = smile_df[col].dropna().tolist()
        smile_texts.extend(texts)

    # 필터링: 위협 키워드 없고 + 길이 20~500자인 것만
    smile_filtered = [
        t for t in smile_texts
        if not contains_threat(t) and 20 < len(str(t)) < 500
    ]

    # 단문 3개씩 합쳐서 멀티턴 대화 형태로 변환
    # → DKTC train 데이터가 평균 9~10턴이 하나로 이어진 형태이므로
    smile_convs = []
    random.shuffle(smile_filtered)
    for i in range(0, len(smile_filtered) - 2, 3):  # 3개씩 묶음
        conv = ' '.join(smile_filtered[i:i+3])
        if 50 < len(conv) < 500:  # DKTC 데이터 길이 범위에 맞춤
            smile_convs.append(conv)

    selected = smile_convs[:400]
    normal_samples.extend(selected)
    print(f"  → {len(selected)}개 수집")

except Exception as e:
    print(f"  SmileStyle 오류: {e}")
    print("  → 건너뜀")

In [ ]:
# ============================================================
# STEP 2-2: 소스 2 - KakaoChatData (목표 300개)
# [Level 2] 실제 카카오톡 대화 스타일 챗봇 데이터 (~73K쌍)
# 선택 이유: 실제 메신저 대화 패턴이 DKTC의 대화체와 가장 자연스럽게 일치
# 처리 방법: Q(질문)+A(답변)를 합치고, 3쌍을 이어 멀티턴 형태로
# ============================================================
print("="*50)
print("소스 2: KakaoChatData 카카오톡 대화")
print("="*50)

try:
    kakao_df = pd.read_csv('ChatbotData.csv')
    print(f"전체 데이터: {len(kakao_df)}개")
    print(f"컬럼: {list(kakao_df.columns)}")

    # Q(질문)와 A(답변)를 합쳐서 1턴 대화로 만들기
    kakao_convs = []
    for _, row in kakao_df.iterrows():
        q = str(row.get('Q', row.iloc[0]))
        a = str(row.get('A', row.iloc[1]))
        conv = f"{q} {a}"  # "오늘 뭐해? 나 집에 있어"
        if not contains_threat(conv) and 20 < len(conv) < 500:
            kakao_convs.append(conv)

    # 3개 QA쌍을 이어 멀티턴 대화로 변환 (DKTC 형태에 맞춤)
    random.shuffle(kakao_convs)
    kakao_multi = []
    for i in range(0, len(kakao_convs) - 2, 3):
        conv = ' '.join(kakao_convs[i:i+3])
        if 80 < len(conv) < 500:
            kakao_multi.append(conv)

    selected = kakao_multi[:300]
    normal_samples.extend(selected)
    print(f"  → {len(selected)}개 수집")

except Exception as e:
    print(f"  KakaoChatData 오류: {e}")
    print("  → 건너뜀")

In [ ]:
# ============================================================
# STEP 2-3: 소스 3 - kor_unsmile 비혐오 문장 (목표 200개)
# [Level 2] 스마일게이트 혐오표현 데이터셋에서 "비혐오(clean=1)"만 추출
# 선택 이유: 혐오 데이터셋의 비혐오 라벨 = "사람이 검증한 무해한 문장"
#           → 일반대화로 사용하기에 신뢰도가 높음
# 처리 방법: 4개 문장을 합쳐서 대화 형태로 (단문이 많아서)
# ============================================================
print("="*50)
print("소스 3: kor_unsmile (비혐오 문장)")
print("="*50)

try:
    # HuggingFace Hub에서 자동 다운로드
    unsmile_ds = load_dataset('smilegate-ai/kor_unsmile', split='train')
    unsmile_df = unsmile_ds.to_pandas()
    print(f"전체: {len(unsmile_df)}개")
    print(f"컬럼: {list(unsmile_df.columns)}")

    # clean=1: 사람이 "비혐오"로 라벨링한 문장만 추출
    if 'clean' in unsmile_df.columns:
        clean_texts = unsmile_df[unsmile_df['clean'] == 1]['문장'].tolist()
    else:
        # clean 컬럼이 없으면: 모든 혐오 라벨이 0인 행 = 비혐오
        label_cols = [c for c in unsmile_df.columns if c not in ['문장', 'clean']]
        clean_mask = unsmile_df[label_cols].sum(axis=1) == 0
        clean_texts = unsmile_df[clean_mask]['문장'].tolist()

    # 위협 키워드 필터 + 길이 필터
    clean_filtered = [
        t for t in clean_texts
        if not contains_threat(t) and 10 < len(str(t)) < 300
    ]

    # 4개 문장씩 합쳐서 대화 형태로 (이 데이터는 단문이 많음)
    random.shuffle(clean_filtered)
    unsmile_convs = []
    for i in range(0, len(clean_filtered) - 3, 4):
        conv = ' '.join(clean_filtered[i:i+4])
        if 50 < len(conv) < 500:
            unsmile_convs.append(conv)

    selected = unsmile_convs[:200]
    normal_samples.extend(selected)
    print(f"  → {len(selected)}개 수집")

except Exception as e:
    print(f"  kor_unsmile 오류: {e}")
    print("  → 건너뜀")

In [ ]:
# ============================================================
# STEP 2-4: 소스 4 - NSMC 네이버 영화리뷰 (목표 100개)
# [Level 2] Ex06에서 다뤘던 NSMC 데이터를 다른 용도로 재활용
# 선택 이유: 긍정 리뷰 중 일상 감정표현("재밌", "대박") = 무해한 일상어
# 처리 방법: 5개 리뷰를 합쳐서 잡담 대화처럼 구성
# 주의: 리뷰 데이터이므로 대화체가 아님 → 보조 소스로만 사용 (100개)
# ============================================================
print("="*50)
print("소스 4: NSMC 네이버 영화리뷰 (긍정+일상)")
print("="*50)

try:
    nsmc_ds = load_dataset('nsmc', split='train')
    nsmc_df = nsmc_ds.to_pandas()
    print(f"전체: {len(nsmc_df)}개")

    # 일상적 긍정 키워드가 포함된 리뷰만 선별
    # → "존나 재밌다" 같은 구어체 표현이 DKTC 스타일과 어느 정도 유사
    daily_keywords = ['재밌', '좋았', '최고', '감동', '웃기', '대박', '꿀잼',
                      '힐링', '따뜻', '행복', '사랑']

    positive = nsmc_df[nsmc_df['label'] == 1]['document'].dropna().tolist()
    daily_reviews = [
        t for t in positive
        if any(kw in str(t) for kw in daily_keywords)  # 일상 키워드 포함
        and not contains_threat(t)                       # 위협 키워드 미포함
        and 15 < len(str(t)) < 200                      # 적절한 길이
    ]

    # 5개 리뷰를 합쳐서 잡담 대화처럼 길이 맞추기
    random.shuffle(daily_reviews)
    nsmc_convs = []
    for i in range(0, len(daily_reviews) - 4, 5):
        conv = ' '.join(daily_reviews[i:i+5])
        if 80 < len(conv) < 500:
            nsmc_convs.append(conv)

    selected = nsmc_convs[:100]
    normal_samples.extend(selected)
    print(f"  → {len(selected)}개 수집")

except Exception as e:
    print(f"  NSMC 오류: {e}")
    print("  → 건너뜀")

In [ ]:
# ============================================================
# STEP 2-5: 소스 5 - 경계 케이스 (25개) ⭐ 핵심 차별화 포인트
# [Level 2] 직접 설계한 "위협처럼 보이지만 실제로는 일반인" 대화
#
# 왜 필요한가?
#   한국어 구어체에서 "야 죽을래 ㅋㅋ", "돈 내놔 ㅋㅋ" 같은 표현은
#   표면적으로 위협 키워드를 포함하지만 실제로는 친구 간 농담임.
#   이런 경계 케이스가 없으면 모델이 키워드만 보고 과잉 분류(false positive)함.
#
# 설계 원칙: 4가지 혼동 패턴을 커버
#   1. 협박 키워드 + 실제 농담 ("죽을래 ㅋㅋ")
#   2. 갈취 키워드 + 실제 친구 부탁 ("천원만 빌려줘")
#   3. 직장 키워드 + 실제 일상 스트레스 ("퇴사하고 싶다 ㅋㅋ")
#   4. 순수 일상 대화 ("치킨 먹을까 피자 먹을까")
# ============================================================
print("="*50)
print("소스 5: 경계 케이스 (위협처럼 보이지만 일반)")
print("="*50)

boundary_cases = [
    # ── 패턴 1: 협박처럼 보이는 농담 ──
    "야 죽을래 ㅋㅋ 아 진짜 웃겨서 죽겠다 아 배아파 ㅋㅋㅋ 진짜 미쳤어 너 개그맨 해라",
    "야 너 진짜 맞을래 ㅋㅋ 아 왜 그런 말을 해서 웃기게 만들어 아 진짜 복근 생기겠다",
    "때려치우고 싶다 뭘 회사 오늘 진짜 힘들었어 야 치킨 먹자 나 오늘 자격 있어",
    "돈 내놔 ㅋㅋ 밥값 네가 쏜다며 아 맞다 내가 쏜다고 했지 ㅋㅋ 어디 갈까",
    "너 진짜 미쳤다 ㅋㅋ 이걸 어떻게 생각해내 와 천재 아니야 대단하다 진짜",

    # ── 패턴 2: 갈취처럼 보이는 친구 대화 ──
    "야 담배 한 개비 줘봐 아 나 오늘 스트레스 받아서 한 대만 ㅋㅋ 고마워 내일 사줄게",
    "야 천원만 빌려줘 자판기 커피 마시고 싶은데 지갑을 놓고 왔어 내일 바로 갚을게",
    "이거 나 좀 줘 뭐 이 과자 맛있어 보여서 하나만 줘봐 오 진짜 맛있다",
    "야 그거 빌려줘 뭘 충전기 배터리 없어서 잠깐만 쓸게 고마워",
    "밥 사라 ㅋㅋ 야 오늘 내 생일인데 당연히 네가 사야지 어디 갈까",

    # ── 패턴 3: 직장 괴롭힘처럼 보이는 일상 스트레스 ──
    "오늘 야근이야 또 아 진짜 힘들다 그래도 이번 프로젝트 끝나면 좀 쉴 수 있겠지",
    "회의 또 해 진짜 오늘만 세번째야 그래도 뭐 좋은 아이디어 나왔으니까 괜찮아",
    "상사가 또 일 줬어 근데 뭐 그래도 인정해주니까 열심히 해야지 파이팅",
    "퇴사하고 싶다 ㅋㅋ 아 농담이야 월급날이니까 참는거지 오늘 뭐 먹을까",
    "야 우리 부장님 또 회식 잡았대 아 귀찮다 그래도 고기니까 ㅋㅋ 가자",

    # ── 패턴 4: 순수 일상 대화 (위협 요소 없음) ──
    "이거 들어봐 와 이 노래 진짜 좋다 그치 요즘 이것만 들어 중독됐어",
    "야 오늘 날씨 진짜 좋다 나가자 어디 갈까 한강 갈까 치킨 시켜서 먹자",
    "게임 할래 뭐 할까 롤 할까 발로란트 할까 아 나 롤 밴당했어 ㅋㅋ 발로 하자",
    "드라마 봤어 뭐 그 어제 나온거 아 진짜 재밌었어 다음주가 기대된다",
    "배고프다 뭐 먹을까 치킨 먹을까 피자 먹을까 둘 다 시킬까 ㅋㅋ 그러자",
    "야 주말에 뭐해 나 아무것도 안해 그러면 놀자 어디 갈까 영화 보러 갈까",
    "시험 망했어 ㅋㅋ 아 그래도 뭐 다음에 잘하면 되지 오늘은 놀자",
    "운동 갈래 같이 헬스장 갈까 아 귀찮은데 그래도 가야지 건강이 최고야",
    "엄마가 용돈 줬어 ㅋㅋ 얼마 5만원 와 부럽다 나도 달라고 해야지",
    "택배 왔다 뭐 시켰어 아 그거 옷 샀어 예쁘지 응 잘 어울린다"
]

normal_samples.extend(boundary_cases)
print(f"  → {len(boundary_cases)}개 추가")

print(f"\n{'='*50}")
print(f"총 합성 일반대화: {len(normal_samples)}개")
print(f"{'='*50}")

In [ ]:
# ============================================================
# STEP 2-6: 합성 데이터를 train에 통합
# [Level 2] 5개 소스에서 수집한 일반대화를 하나의 DataFrame으로 합침
# → 원본 train(3,950개, 4클래스) + 합성 일반대화(~1000개) = train_full
# ============================================================

# 합성 데이터를 DataFrame으로 변환
normal_df = pd.DataFrame({
    'idx': [f'n_{i:03d}' for i in range(len(normal_samples))],  # n_000, n_001, ...
    'class': '일반 대화',                                         # 라벨은 "일반 대화"
    'conversation': normal_samples
})

print(f"합성 일반대화 DataFrame: {len(normal_df)}개")
print(f"샘플:")
for i in range(min(3, len(normal_df))):
    print(f"  [{i}] {normal_df.iloc[i]['conversation'][:80]}...")

# 원본 train과 합성 데이터를 concat
# → 이제 5개 클래스가 모두 존재하는 완전한 학습 데이터셋!
train_full = pd.concat([train_df[['idx', 'class', 'conversation']], normal_df],
                       ignore_index=True)
print(f"\n통합 train: {len(train_full)}개")
print(f"\n클래스 분포 (이제 5개 클래스 모두 있음!):")
print(train_full['class'].value_counts())

## STEP 3: 전처리 & 토크나이저
> **Level 1 (베이스라인)** — Ex06(NSMC 감성분석) + Ex07(SentencePiece 토큰화)에서 배운 패턴
>
> - Ex06: 텍스트 정제 함수 패턴 (정규식, 특수문자 제거)
> - Ex07: 한국어 토큰화 원리 이해 → ELECTRA의 WordPiece가 구어체를 어떻게 처리하는지 파악
> - **핵심**: 구어체/반말 특성(ㅋㅋ, ㅠㅠ 등)을 보존하면서 노이즈만 제거

In [ ]:
# ============================================================
# STEP 3-1: 텍스트 전처리 함수
# [Level 1] Ex06에서 배운 텍스트 정제 패턴 적용
#
# 설계 원칙: "최소한의 전처리"
#   DKTC 데이터는 반말+구어체 → "ㅋㅋ", "ㅠㅠ", "~" 등이 의미를 가짐
#   예: "죽을래 ㅋㅋ" (농담) vs "죽을래" (위협) → ㅋㅋ를 제거하면 구분 불가!
#   따라서 구어체 요소는 보존하고, 의미없는 특수문자만 제거
# ============================================================
import re

def preprocess(text):
    """텍스트 전처리 - 구어체 보존, 노이즈만 제거"""
    text = str(text)
    text = re.sub(r'\s+', ' ', text)  # 연속 공백 → 공백 1개로 통일
    # 한글(가-힣) + 영숫자 + 자음/모음(ㅋㅎㅠㅜ) + 기본 문장부호만 보존
    # → ㅋ, ㅎ, ㅠ, ㅜ는 감정 표현이므로 반드시 보존!
    text = re.sub(r'[^가-힣a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ\s,.!?~ㅋㅎㅠㅜ]', '', text)
    return text.strip()

# train과 test 모두에 동일한 전처리 적용 (일관성 중요!)
train_full['conversation'] = train_full['conversation'].apply(preprocess)
test_df['conversation'] = test_df['conversation'].apply(preprocess)

# 문자열 클래스명 → 정수 라벨로 변환 (모델 학습에 필요)
# "협박 대화" → 0, "갈취 대화" → 1, ... "일반 대화" → 4
train_full['label'] = train_full['class'].map(CLASS2IDX)

# NaN 체크: 매핑 안 된 클래스가 있으면 코드 오류
print(f"라벨 NaN 수: {train_full['label'].isna().sum()}")
if train_full['label'].isna().sum() > 0:
    print("⚠️ 매핑 안 된 클래스:")
    print(train_full[train_full['label'].isna()]['class'].unique())

train_full['label'] = train_full['label'].astype(int)
print(f"\n라벨 분포:")
print(train_full['label'].value_counts().sort_index())
print(f"\n전처리 샘플:")
print(train_full.iloc[0]['conversation'][:100])

In [ ]:
# ============================================================
# STEP 3-2: KcELECTRA 토크나이저 로드 + 토큰 길이 분석
# [Level 1] Ex07(SentencePiece)에서 배운 토큰화 개념 활용
# [Level 2] Ex09(Transformer)에서 배운 Pre-trained 모델 활용
#
# KcELECTRA 토크나이저 = WordPiece 방식
#   Ex07에서 배운 SentencePiece와 유사한 서브워드 토큰화
#   "죽겠다" → ["죽", "##겠", "##다"] 처럼 분리
#   한국어 댓글/구어체로 학습된 vocab이므로 "ㅋㅋ", "ㅠㅠ" 등도 잘 처리
#
# MAX_LEN 검증: 95%ile 이하면 대부분의 데이터가 잘리지 않음
# ============================================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"토크나이저: {MODEL_NAME}")
print(f"Vocab size: {tokenizer.vocab_size}")

# 전체 데이터의 토큰 길이 분포 확인
# → MAX_LEN=256이 적절한지 데이터로 검증
token_lens = []
for text in train_full['conversation'].values:
    tokens = tokenizer.encode(text, add_special_tokens=True)  # [CLS] + tokens + [SEP]
    token_lens.append(len(tokens))

print(f"\n토큰 길이 통계:")
print(f"  평균: {np.mean(token_lens):.1f}")
print(f"  최대: {np.max(token_lens)}")
print(f"  95%ile: {np.percentile(token_lens, 95):.0f}")
# 95% 이상 커버되면 MAX_LEN이 적절한 것
print(f"  MAX_LEN={MAX_LEN}으로 커버되는 비율: {sum(1 for l in token_lens if l <= MAX_LEN)/len(token_lens)*100:.1f}%")

## STEP 4: Dataset, Focal Loss, R-Drop 정의
> **Level 2 (최신 논문 기법 적용)** — 3개의 핵심 컴포넌트 정의
>
> | 컴포넌트 | 논문 | 왜 적용했나 |
> |----------|------|-------------|
> | DKTCDataset | Ex06 NSMCDataset 패턴 | PyTorch Dataset 구조 (\_\_getitem\_\_ 패턴) |
> | **Focal Loss** | Lin et al., 2017 "Focal Loss for Dense Object Detection" | 합성 후에도 일반대화 비율이 다름 → 소수 클래스에 더 큰 가중치 |
> | **R-Drop** | Liang et al., 2021 "R-Drop: Regularized Dropout for Neural Networks" | ~5K 소규모 데이터 → 과적합 위험 → 드롭아웃 정규화로 방지 |

In [ ]:
# ============================================================
# STEP 4: Dataset, FocalLoss, R-Drop 정의
# ============================================================

# ──────────────────────────────────────────────
# (1) DKTCDataset: PyTorch Dataset 클래스
# [Level 1] Ex06의 NSMCDataset 패턴을 그대로 활용
#   Ex06에서 배운 것: __getitem__에서 토크나이저 호출 → 텐서 반환
#   달라진 점: NSMC는 2클래스(긍정/부정), DKTC는 5클래스
# ──────────────────────────────────────────────
class DKTCDataset(Dataset):
    """
    DKTC 데이터셋 (Ex06 NSMCDataset 패턴)
    텍스트 → 토크나이저 → input_ids + attention_mask + labels
    """
    def __init__(self, texts, labels=None, tokenizer=None, max_len=256):
        self.texts = texts
        self.labels = labels       # None이면 test 데이터 (라벨 없음)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        # 토크나이저가 텍스트를 토큰 ID로 변환
        # padding='max_length': 모든 샘플을 같은 길이로 맞춤 (배치 처리용)
        # truncation=True: MAX_LEN 초과 시 자름
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(0),       # [MAX_LEN] 토큰 ID
            'attention_mask': encoding['attention_mask'].squeeze(0),  # [MAX_LEN] 패딩=0, 실제=1
        }
        # ELECTRA는 token_type_ids도 사용 (BERT와 동일)
        if 'token_type_ids' in encoding:
            item['token_type_ids'] = encoding['token_type_ids'].squeeze(0)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


# ──────────────────────────────────────────────
# (2) Focal Loss: 클래스 불균형 해결
# [Level 2] 논문: Lin et al., 2017 "Focal Loss for Dense Object Detection"
#
# 왜 적용했나?
#   합성 후에도 클래스 비율이 다름 (협박 896 vs 일반 ~1000)
#   CE Loss는 모든 샘플에 동일한 가중치 → 다수 클래스에 편향
#   Focal Loss는 "이미 잘 맞추는 쉬운 샘플"의 가중치를 줄이고
#   "틀리기 쉬운 어려운 샘플"에 집중 → 소수 클래스 성능 향상
#
# 수식: FL(pt) = -alpha * (1-pt)^gamma * log(pt)
#   pt = 모델이 정답에 부여한 확률
#   gamma = 2.0 (기본값): pt가 높으면 (1-pt)^2 ≈ 0 → 쉬운 샘플 무시
#   alpha = 클래스별 가중치 (역빈도 기반)
# ──────────────────────────────────────────────
class FocalLoss(nn.Module):
    """Focal Loss (Lin et al., 2017)"""
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha   # 클래스별 가중치 텐서 (역빈도 기반으로 계산)
        self.gamma = gamma   # focusing parameter: 클수록 어려운 샘플에 더 집중
        self.reduction = reduction

    def forward(self, inputs, targets):
        # 1단계: 일반 CE Loss 계산 (클래스 가중치 적용)
        ce_loss = F.cross_entropy(inputs, targets, weight=self.alpha, reduction='none')
        # 2단계: pt = 정답 클래스의 예측 확률
        pt = torch.exp(-ce_loss)  # exp(-CE) = 정답 확률
        # 3단계: Focal 가중치 적용 → 쉬운 샘플(pt 높음)은 가중치 ↓
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss


# ──────────────────────────────────────────────
# (3) R-Drop: 드롭아웃 정규화
# [Level 2] 논문: Liang et al., 2021 "R-Drop: Regularized Dropout"
#
# 왜 적용했나?
#   데이터가 ~5K로 소규모 → Pre-trained 모델도 과적합 위험
#   R-Drop 원리: 같은 입력을 2번 forward pass (드롭아웃이 다르게 적용됨)
#   → 두 출력이 비슷해지도록 KL Divergence를 추가 loss로 사용
#   → 드롭아웃에 의한 "노이즈"에 덜 민감해짐 = 일반화 능력 향상
#
# 수식: L = CE_loss + alpha * KL_div(output1 || output2)
#   alpha = 0.7 (KL loss의 가중치, 논문 권장값)
# ──────────────────────────────────────────────
def compute_rdrop_loss(logits1, logits2, labels, loss_fn, alpha=0.7):
    """
    R-Drop Loss (Liang et al., 2021)
    같은 입력 2번 forward → CE 평균 + alpha * 양방향 KL Divergence
    """
    # CE/Focal loss 2개의 평균
    loss1 = loss_fn(logits1, labels)
    loss2 = loss_fn(logits2, labels)
    ce_loss = (loss1 + loss2) / 2

    # 양방향 KL Divergence: P→Q와 Q→P 모두 계산하여 대칭성 확보
    p = F.log_softmax(logits1, dim=-1)  # 첫 번째 forward 결과의 log 확률
    q = F.log_softmax(logits2, dim=-1)  # 두 번째 forward 결과의 log 확률
    kl_loss = (
        F.kl_div(p, q.exp(), reduction='batchmean') +  # KL(Q || P)
        F.kl_div(q, p.exp(), reduction='batchmean')     # KL(P || Q)
    ) / 2

    # 최종 loss = 분류 loss + alpha * 정규화 loss
    return ce_loss + alpha * kl_loss


print("Dataset, FocalLoss, R-Drop 정의 완료")

## STEP 5: Ablation 실험 실행
> **Level 3 (Ablation Study로 정량 검증)** — Ex03(CIFAR-10 Ablation Study)에서 배운 실험 설계
>
> **Ablation Study란?** (Ex03에서 배운 핵심 개념)
> - 모델에 적용한 기법을 하나씩 빼거나 추가하면서 **각 기법의 기여도**를 측정
> - "이 기법이 정말 도움이 되는가?"를 **숫자로 증명**
>
> | 실험 | Loss | R-Drop | 합성데이터 | 비교 포인트 |
> |------|------|--------|-----------|------------|
> | Exp1 | CE Loss | X | 전체 | **baseline** (비교 기준점) |
> | Exp2 | **Focal** Loss | X | 전체 | Exp1 대비 F1 ↑ → **불균형 해결 효과** 증명 |
> | Exp3 | Focal Loss | **O** (α=0.7) | 전체 | Exp2 대비 F1 ↑ → **과적합 방지 효과** 증명 |
> | Exp4 | Focal Loss | O | **500개 축소** | Exp3 대비 F1 ↓ → **데이터 양의 중요성** 증명 |

In [ ]:
# ============================================================
# STEP 5-1: 학습/검증 함수 정의
# [Level 1] Ex06에서 배운 학습 루프 패턴을 확장
#   Ex06: 단순 for epoch → for batch → loss.backward()
#   확장: R-Drop 분기, gradient clipping, scheduler 추가
# ============================================================

def train_one_epoch(model, dataloader, optimizer, scheduler, loss_fn,
                    use_rdrop=False, rdrop_alpha=0.7):
    """
    1 epoch 학습
    use_rdrop=True일 때: 같은 배치를 2번 forward → R-Drop loss 적용
    """
    model.train()  # 학습 모드 (드롭아웃 활성화 → R-Drop의 핵심!)
    total_loss = 0
    all_preds, all_labels = [], []

    for batch in dataloader:
        # GPU로 데이터 이동
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        model_kwargs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        if 'token_type_ids' in batch:
            model_kwargs['token_type_ids'] = batch['token_type_ids'].to(DEVICE)

        if use_rdrop:
            # [Level 2] R-Drop: 같은 입력으로 2번 forward
            # → 드롭아웃이 랜덤이므로 출력이 약간 다름
            # → 이 차이를 줄이도록 학습 = 더 안정적인 모델
            outputs1 = model(**model_kwargs)  # 1번째 forward (드롭아웃 A)
            outputs2 = model(**model_kwargs)  # 2번째 forward (드롭아웃 B)
            loss = compute_rdrop_loss(
                outputs1.logits, outputs2.logits, labels,
                loss_fn, alpha=rdrop_alpha
            )
            logits = outputs1.logits
        else:
            # 일반 학습: 1번 forward
            outputs = model(**model_kwargs)
            logits = outputs.logits
            loss = loss_fn(logits, labels)

        # 역전파 + 최적화
        optimizer.zero_grad()                                       # 기울기 초기화
        loss.backward()                                             # 역전파
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)  # 기울기 폭발 방지
        optimizer.step()                                            # 가중치 업데이트
        scheduler.step()                                            # 학습률 조정

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=-1)  # 가장 높은 확률의 클래스 = 예측
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')  # macro: 클래스별 F1의 평균
    return avg_loss, acc, f1


def evaluate(model, dataloader, loss_fn):
    """검증 (드롭아웃 비활성화 → 결정론적 출력)"""
    model.eval()  # 평가 모드 (드롭아웃 OFF)
    total_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():  # 기울기 계산 불필요 → 메모리 절약
        for batch in dataloader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            model_kwargs = {'input_ids': input_ids, 'attention_mask': attention_mask}
            if 'token_type_ids' in batch:
                model_kwargs['token_type_ids'] = batch['token_type_ids'].to(DEVICE)

            outputs = model(**model_kwargs)
            logits = outputs.logits
            loss = loss_fn(logits, labels)

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return avg_loss, acc, f1, all_preds, all_labels


print("학습/검증 함수 정의 완료")

In [ ]:
# ============================================================
# STEP 5-2: 실험 실행 함수 (Ablation 공통 프레임워크)
# [Level 3] Ex03에서 배운 Ablation Study 패턴
#   핵심: 모든 실험이 동일한 함수를 사용 → 공정한 비교 보장
#   다른 것: loss_fn, use_rdrop, 데이터만 변경
# ============================================================

def run_experiment(exp_name, train_data, val_data, loss_fn,
                   use_rdrop=False, rdrop_alpha=0.7, epochs=EPOCHS):
    """
    Ablation 실험 실행 함수
    → 동일 구조에서 loss/R-Drop/데이터만 바꿔가며 공정 비교
    Returns: history + best model state + validation 예측 결과
    """
    print(f"\n{'='*60}")
    print(f"  {exp_name}")
    print(f"  Loss: {type(loss_fn).__name__}, R-Drop: {use_rdrop}, Data: {len(train_data)}")
    print(f"{'='*60}")

    # ── 데이터 준비 ──
    train_dataset = DKTCDataset(
        train_data['conversation'].values,
        train_data['label'].values,
        tokenizer, MAX_LEN
    )
    val_dataset = DKTCDataset(
        val_data['conversation'].values,
        val_data['label'].values,
        tokenizer, MAX_LEN
    )
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # ── 모델: 매 실험마다 새로 초기화 (공정 비교!) ──
    # [Level 2] Ex09에서 배운 Pre-trained 모델 Fine-tuning
    # AutoModelForSequenceClassification: ELECTRA + 분류 head(5클래스)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=NUM_CLASSES
    ).to(DEVICE)

    # ── 옵티마이저: AdamW (BERT 계열 표준) ──
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY
    )
    # ── 스케줄러: linear warmup → 선형 감소 ──
    # 처음 10% 스텝은 LR을 0→2e-5로 서서히 올림 (안정적 시작)
    # 이후 선형으로 감소 → 수렴 안정성
    total_steps = len(train_loader) * epochs
    warmup_steps = int(total_steps * WARMUP_RATIO)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, warmup_steps, total_steps
    )

    # ── 학습 루프 ──
    history = {
        'train_loss': [], 'train_acc': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_f1': []
    }
    best_val_f1 = 0
    best_state = None

    for epoch in range(epochs):
        train_loss, train_acc, train_f1 = train_one_epoch(
            model, train_loader, optimizer, scheduler, loss_fn,
            use_rdrop=use_rdrop, rdrop_alpha=rdrop_alpha
        )
        val_loss, val_acc, val_f1, _, _ = evaluate(model, val_loader, loss_fn)

        # 이력 저장 (STEP 6 시각화용)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_f1'].append(train_f1)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)

        # Best 모델 저장: Val F1이 가장 높은 epoch의 가중치 보관
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

        print(f"  Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} F1: {train_f1:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}"
              f"{' ★' if val_f1 >= best_val_f1 else ''}")

    print(f"  → Best Val F1: {best_val_f1:.4f}")

    # Best 모델로 최종 검증 (Confusion Matrix용 예측값 수집)
    model.load_state_dict(best_state)
    model.to(DEVICE)
    _, _, _, val_preds, val_labels = evaluate(model, val_loader, loss_fn)

    # GPU 메모리 정리 (다음 실험을 위해)
    del model
    torch.cuda.empty_cache()

    return {
        'name': exp_name,
        'history': history,
        'best_val_f1': best_val_f1,
        'best_state': best_state,
        'val_preds': val_preds,
        'val_labels': val_labels
    }

print("실험 함수 정의 완료")

In [ ]:
# ============================================================
# STEP 5-3: 데이터 분할 (Stratified Split)
# [Level 1] 검증셋 구성 - 클래스 비율을 유지하는 분할
#   stratify: 원본 비율 그대로 train/val에 분배
#   → 검증셋에도 5개 클래스가 고르게 포함되어야 공정한 평가
# ============================================================

# 전체 데이터 분할 (Exp1~3용)
train_split, val_split = train_test_split(
    train_full, test_size=VAL_RATIO,       # 15% 검증셋
    stratify=train_full['label'],           # 클래스 비율 유지!
    random_state=SEED                       # 재현성
)
print(f"Train split: {len(train_split)}, Val split: {len(val_split)}")
print(f"\nTrain 라벨 분포:")
print(train_split['label'].value_counts().sort_index())
print(f"\nVal 라벨 분포:")
print(val_split['label'].value_counts().sort_index())

# ── Exp4용 축소 데이터 ──
# [Level 3] Ablation: 합성 일반대화를 500개로 줄여서 "데이터 양의 영향" 측정
# → Exp3(전체) vs Exp4(축소) 비교 → 데이터가 많을수록 좋다는 것을 증명
normal_500 = train_full[train_full['label'] == 4].sample(
    n=min(500, len(train_full[train_full['label']==4])),
    random_state=SEED
)
original_data = train_full[train_full['label'] != 4]  # 원본 4클래스는 그대로
train_reduced = pd.concat([original_data, normal_500], ignore_index=True)
train_reduced_split, val_reduced_split = train_test_split(
    train_reduced, test_size=VAL_RATIO, stratify=train_reduced['label'],
    random_state=SEED
)
print(f"\nExp4 축소 데이터: {len(train_reduced)}개")

In [ ]:
# ============================================================
# STEP 5-4: 클래스 가중치 계산 (Focal Loss의 alpha 파라미터)
# [Level 2] 역빈도(inverse frequency) 방식
#   클래스별 가중치 = 전체 샘플 수 / (클래스 수 × 해당 클래스 샘플 수)
#   → 적은 클래스일수록 가중치 ↑ → Focal Loss가 소수 클래스에 더 집중
# ============================================================
label_counts = train_split['label'].value_counts().sort_index()
total = len(train_split)
class_weights = torch.tensor(
    [total / (NUM_CLASSES * count) for count in label_counts.values],
    dtype=torch.float32
).to(DEVICE)
print(f"클래스 가중치: {class_weights}")
# 예: 일반대화가 적으면 가중치 > 1.0, 많으면 < 1.0

In [ ]:
# ============================================================
# [Level 3] Ablation Exp1: CE Loss (Baseline)
# ============================================================
# 비교 기준점(baseline): 가장 기본적인 설정
# → CE Loss + R-Drop 없음
# → 이 결과와 비교해서 Focal Loss, R-Drop의 효과를 측정
# ============================================================
ce_loss_fn = nn.CrossEntropyLoss().to(DEVICE)
result1 = run_experiment(
    'Exp1: CE Loss (Baseline)',
    train_split, val_split, ce_loss_fn,
    use_rdrop=False  # R-Drop 없음
)

In [ ]:
# ============================================================
# [Level 3] Ablation Exp2: Focal Loss (불균형 해결 효과 측정)
# ============================================================
# 변경: CE Loss → Focal Loss (Lin et al., 2017)
# 비교: Exp1(CE) vs Exp2(Focal) → F1 차이 = Focal Loss의 기여도
# 기대: 소수 클래스(일반대화) F1 ↑, 전체 macro F1 ↑
# ============================================================
focal_loss_fn = FocalLoss(alpha=class_weights, gamma=2.0).to(DEVICE)
result2 = run_experiment(
    'Exp2: Focal Loss',
    train_split, val_split, focal_loss_fn,
    use_rdrop=False  # 아직 R-Drop 없음 (하나만 변경해서 비교)
)

In [ ]:
# ============================================================
# [Level 3] Ablation Exp3: Focal Loss + R-Drop (과적합 방지 효과 측정)
# ============================================================
# 변경: R-Drop 추가 (Liang et al., 2021)
# 비교: Exp2(Focal만) vs Exp3(Focal+R-Drop) → F1 차이 = R-Drop의 기여도
# 기대: train-val F1 gap 감소 (과적합 완화), val F1 ↑
# ============================================================
focal_loss_fn3 = FocalLoss(alpha=class_weights, gamma=2.0).to(DEVICE)
result3 = run_experiment(
    'Exp3: Focal + R-Drop (Full Data)',
    train_split, val_split, focal_loss_fn3,
    use_rdrop=True, rdrop_alpha=0.7  # R-Drop ON, alpha=0.7 (논문 권장)
)

In [ ]:
# ============================================================
# [Level 3] Ablation Exp4: 축소 데이터 (합성 데이터 양의 영향 측정)
# ============================================================
# 변경: 합성 일반대화를 ~1000개 → 500개로 축소
# 비교: Exp3(전체) vs Exp4(축소) → F1 차이 = 합성 데이터 양의 중요성
# 기대: 데이터가 줄면 F1 ↓ → "5개 소스 통합"의 가치 증명
# ============================================================

# 축소 데이터용 클래스 가중치 재계산 (비율이 달라졌으므로)
label_counts_r = train_reduced_split['label'].value_counts().sort_index()
total_r = len(train_reduced_split)
class_weights_r = torch.tensor(
    [total_r / (NUM_CLASSES * count) for count in label_counts_r.values],
    dtype=torch.float32
).to(DEVICE)

focal_loss_fn4 = FocalLoss(alpha=class_weights_r, gamma=2.0).to(DEVICE)
result4 = run_experiment(
    'Exp4: Focal + R-Drop (Reduced Data)',
    train_reduced_split, val_reduced_split, focal_loss_fn4,
    use_rdrop=True, rdrop_alpha=0.7
)

# 모든 실험 결과를 리스트로 모음 (STEP 6 시각화용)
results = [result1, result2, result3, result4]
print("\n" + "="*60)
print("  모든 Ablation 실험 완료!")
print("="*60)

## STEP 6: 결과 시각화
> **Level 3 (Ablation 결과 정량 검증)** — Ex03 시각화 패턴 적용
>
> 3가지 시각화로 각 기법의 기여도를 **눈으로 확인**:
> 1. **결과 요약 테이블**: F1/Acc 숫자 비교
> 2. **학습 곡선 (4개 그래프)**: 과적합 여부, 수렴 패턴 확인
> 3. **Confusion Matrix (4개)**: 어떤 클래스를 혼동하는지 확인

In [ ]:
# ============================================================
# STEP 6-1: Ablation 결과 요약 테이블
# [Level 3] 각 기법의 기여도를 숫자로 비교
#   읽는 법: Exp1→Exp2 F1 차이 = Focal Loss 효과
#           Exp2→Exp3 F1 차이 = R-Drop 효과
#           Exp3→Exp4 F1 차이 = 데이터 양 효과
# ============================================================
print("\n" + "="*70)
print("  Ablation Study 결과 요약")
print("="*70)
print(f"{'실험':<35} {'Best Val F1':>12} {'Best Val Acc':>12}")
print("-"*70)
for r in results:
    best_acc = max(r['history']['val_acc'])
    print(f"{r['name']:<35} {r['best_val_f1']:>12.4f} {best_acc:>12.4f}")
print("-"*70)

# 가장 좋은 실험 자동 선택 (F1 기준)
best_exp = max(results, key=lambda x: x['best_val_f1'])
print(f"\n★ Best: {best_exp['name']} (F1={best_exp['best_val_f1']:.4f})")

In [ ]:
# ============================================================
# STEP 6-2: 학습 곡선 시각화
# [Level 3] Ex03에서 배운 Ablation 시각화 패턴
#   4개 그래프: Val F1, Val Acc, Train Loss, Val Loss
#   → 과적합 확인: Train Loss ↓ 인데 Val Loss ↑ 이면 과적합
#   → R-Drop 효과: Exp3이 Exp2보다 train-val gap이 작으면 성공
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6']  # 빨/파/초/보

# 좌상: Validation F1 (가장 중요한 지표!)
for i, r in enumerate(results):
    axes[0,0].plot(r['history']['val_f1'], label=r['name'], color=colors[i], marker='o')
axes[0,0].set_title('Validation F1 Score (Main Metric)')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('F1')
axes[0,0].legend(fontsize=8)
axes[0,0].grid(True, alpha=0.3)

# 우상: Validation Accuracy
for i, r in enumerate(results):
    axes[0,1].plot(r['history']['val_acc'], label=r['name'], color=colors[i], marker='o')
axes[0,1].set_title('Validation Accuracy')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('Accuracy')
axes[0,1].legend(fontsize=8)
axes[0,1].grid(True, alpha=0.3)

# 좌하: Training Loss (과적합 모니터링용)
for i, r in enumerate(results):
    axes[1,0].plot(r['history']['train_loss'], label=r['name'], color=colors[i], marker='o')
axes[1,0].set_title('Training Loss')
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('Loss')
axes[1,0].legend(fontsize=8)
axes[1,0].grid(True, alpha=0.3)

# 우하: Validation Loss (과적합 = 이게 올라감)
for i, r in enumerate(results):
    axes[1,1].plot(r['history']['val_loss'], label=r['name'], color=colors[i], marker='o')
axes[1,1].set_title('Validation Loss')
axes[1,1].set_xlabel('Epoch')
axes[1,1].set_ylabel('Loss')
axes[1,1].legend(fontsize=8)
axes[1,1].grid(True, alpha=0.3)

plt.suptitle('Ablation Study: Learning Curves', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('ablation_learning_curves.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# STEP 6-3: Confusion Matrix 시각화
# [Level 3] 4개 실험의 혼동 행렬을 나란히 비교
#   읽는 법: 대각선 = 정답, 비대각선 = 오분류
#   주목 포인트: Normal(일반) 행/열의 변화
#   → Exp1에서 Normal 오분류 많았다면 → Exp3에서 줄었는지 확인
# ============================================================
fig, axes = plt.subplots(1, 4, figsize=(24, 5))

# 영문 약자로 표시 (공간 절약)
short_names = ['Threat', 'Extort', 'Work', 'Other', 'Normal']

for i, r in enumerate(results):
    cm = confusion_matrix(r['val_labels'], r['val_preds'])
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
        xticklabels=short_names, yticklabels=short_names
    )
    axes[i].set_title(f"{r['name']}\nF1={r['best_val_f1']:.4f}", fontsize=10)
    axes[i].set_ylabel('True')      # 세로 = 실제 라벨
    axes[i].set_xlabel('Predicted')  # 가로 = 모델 예측

plt.suptitle('Confusion Matrices (Validation Set)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('ablation_confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# STEP 6-4: Best 모델 상세 리포트
# [Level 3] 클래스별 Precision/Recall/F1 확인
#   → 어떤 클래스가 약한지 파악 가능
#   → 발표 시 "일반 대화 F1이 XX로, 합성데이터가 효과적임을 증명" 근거
# ============================================================
print(f"\nBest 모델: {best_exp['name']}")
print(f"\nClassification Report:")
print(classification_report(
    best_exp['val_labels'], best_exp['val_preds'],
    target_names=CLASS_NAMES,
    digits=4
))

## STEP 7: Test 예측 & 제출파일 생성
> Best 모델로 test 500개를 예측하고 `submission_final.csv` 생성

In [ ]:
# ============================================================
# STEP 7-1: Best 모델로 Test 예측
# Ablation에서 가장 F1이 높은 모델의 가중치를 불러와서 예측
# ============================================================
print(f"Best 모델: {best_exp['name']}")
print(f"Best Val F1: {best_exp['best_val_f1']:.4f}")

# Best 모델 가중치 로드
model_final = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=NUM_CLASSES
).to(DEVICE)
model_final.load_state_dict(best_exp['best_state'])  # Ablation best 가중치
model_final.eval()                                     # 평가 모드 (드롭아웃 OFF)

# Test 데이터셋 (라벨 없음)
test_dataset = DKTCDataset(
    test_df['conversation'].values,
    labels=None,         # test는 정답 라벨이 없음
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 예측 실행
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)

        model_kwargs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        if 'token_type_ids' in batch:
            model_kwargs['token_type_ids'] = batch['token_type_ids'].to(DEVICE)

        outputs = model_final(**model_kwargs)
        preds = torch.argmax(outputs.logits, dim=-1)  # 가장 높은 확률의 클래스
        all_preds.extend(preds.cpu().numpy())

print(f"\n예측 완료: {len(all_preds)}개")
print(f"\n예측 분포:")
pred_counts = Counter(all_preds)
for label_idx in sorted(pred_counts.keys()):
    print(f"  {IDX2CLASS[label_idx]}: {pred_counts[label_idx]}개")

In [ ]:
# ============================================================
# STEP 7-2: 제출파일 생성 + 예측 품질 검증
# submission_final.csv: idx(t_000~) + class(0~4 정수)
# ============================================================
submission_df['class'] = all_preds
submission_df['class'] = submission_df['class'].astype(int)

submission_df.to_csv('submission_final.csv', index=False)

print("submission_final.csv 저장 완료!")
print(f"\n미리보기:")
print(submission_df.head(10))
print(f"\n클래스 분포:")
print(submission_df['class'].value_counts().sort_index())

# ── 품질 검증: 일반대화(4) 비율이 적절한지 체크 ──
# test 500개 중 일반대화는 ~100개(20%)로 추정
normal_count = sum(1 for p in all_preds if p == 4)
print(f"\n일반 대화 예측: {normal_count}개 ({normal_count/len(all_preds)*100:.1f}%)")
if normal_count < 50:
    print("⚠️ 일반대화 예측이 너무 적습니다. 합성데이터 품질을 확인하세요.")
elif normal_count > 200:
    print("⚠️ 일반대화 예측이 너무 많습니다. 모델이 과도하게 일반으로 분류하고 있습니다.")
else:
    print("일반 대화 비율이 적절합니다 (test 500개 중 ~100개 추정)")

In [ ]:
# ============================================================
# STEP 7-3: 예측 샘플 확인 (사람이 눈으로 검증)
# → 일반대화로 예측된 것이 실제로 일반적인지 확인
# → 위협으로 예측된 것이 실제로 위협적인지 확인
# ============================================================
print("\n" + "="*60)
print("  일반대화(4)로 예측된 test 샘플 확인")
print("="*60)
normal_indices = [i for i, p in enumerate(all_preds) if p == 4]
for idx in normal_indices[:10]:  # 최대 10개만 출력
    text = test_df.iloc[idx]['conversation'][:100]
    print(f"  [{test_df.iloc[idx]['idx']}] {text}...")

print(f"\n위협으로 예측된 샘플도 확인:")
for label_idx in range(4):  # 0~3: 위협 클래스
    indices = [i for i, p in enumerate(all_preds) if p == label_idx]
    if indices:
        i = indices[0]
        text = test_df.iloc[i]['conversation'][:80]
        print(f"  [{IDX2CLASS[label_idx]}] {text}...")

## STEP 8: 프로젝트 정리 & 발표 포인트
> 최종 요약: 전략별 성과, 논문 근거, 학습 이력 활용을 한눈에 정리

In [ ]:
# ============================================================
# STEP 8: 프로젝트 정리 - 발표용 요약
# ============================================================
print("\n" + "="*60)
print("  DKTC 프로젝트 최종 정리")
print("="*60)

print(f"\n[모델] {MODEL_NAME}")
print(f"[데이터] 원본 {len(train_df)}개 + 합성 일반대화 {len(normal_samples)}개 = {len(train_full)}개")
print(f"[합성 소스] SmileStyle, KakaoChatData, kor_unsmile, NSMC, 경계케이스")

print(f"\n[Ablation 결과]")
for r in results:
    print(f"  {r['name']}: F1={r['best_val_f1']:.4f}")

print(f"\n[Best] {best_exp['name']}: F1={best_exp['best_val_f1']:.4f}")
print(f"[제출] submission_final.csv ({len(all_preds)}개 예측)")

# ── 발표 포인트 ──
print(f"\n{'='*60}")
print("  발표 포인트 (가브리엘 페터슨 탑다운 학습법)")
print(f"{'='*60}")
print(f"\n  [Level 1] 문제 발견: train에 일반대화 0개 → EDA에서 즉시 발견")
print(f"  [Level 2] 최신 기법 적용:")
print(f"    - KcELECTRA: Clark et al. (2020) - 한국어 구어체에 최적화된 사전학습")
print(f"    - Focal Loss: Lin et al. (2017) - 클래스 불균형 해결")
print(f"    - R-Drop: Liang et al. (2021) - 소규모 데이터 과적합 방지")
print(f"    - 합성데이터 5개 소스: AugGPT (Dai et al., 2023) 영감")
print(f"  [Level 3] Ablation으로 정량 증명: 각 기법의 F1 기여도를 숫자로 보여줌")

print(f"\n  [학습 이력 활용]")
print(f"    - Ex03: Ablation Study 설계 방법론 → STEP 5 실험 설계")
print(f"    - Ex06: 텍스트 전처리, Dataset 구조 → STEP 3, 4")
print(f"    - Ex07: 한국어 토큰화 이해 → STEP 3 토큰 분석")
print(f"    - Ex09: Transformer Fine-tuning → STEP 4 모델 구성")

print(f"\n  [핵심 차별화]")
print(f"    1. 경계 케이스 25개 - '죽을래 ㅋㅋ' 같은 농담을 일반대화로 학습")
print(f"    2. 5개 소스 통합 - 단일 소스가 아닌 다양한 구어체 데이터")
print(f"    3. Ablation으로 정량 증명 - 각 기법의 기여도를 숫자로 보여줌")

print(f"\n✅ 완료!")

In [ ]:
# Colab에서 제출파일 다운로드
from google.colab import files
files.download('submission_final.csv')